In [18]:
import pandas as pd
from requests import get, codes as http_code
from ratelimit import limits
import time
import json
import os
# from helpers import get_col_names

home_dir="../"

In [19]:
@limits(calls=1, period=1)  # Only one call per second
def download_and_save_snp(rs_id: str):
    # rs_id should be without rs_ keyword, only the number. 
    url = f"https://api.ncbi.nlm.nih.gov/variation/v0/refsnp/{rs_id}"
    out_filepath = home_dir+f"data/dbsnp/reference_snps/{rs_id}.json"
    if os.path.exists: return True
    
    attempt = 0
    while (attempt < 3):
        attempt += 1
        reply = get(url)
        if reply.status_code != http_code.ok:
            print(f"Received error from server {reply.status_code}")
            print(f"Attempt {attempt} of 3")
            time.sleep(10)
            
        # print(reply)
        content_type = reply.headers['content-type']
        if 'application/json' not in content_type:
            raise Exception("Unexpected content type: {}\n{}".format(content_type, url))
        
        
        with open(out_filepath, "w") as f:
            f.write(json.dumps(reply.json()))
        return True

download_and_save_snp("229969")

True

In [28]:
rs_id = "229969"
inp_filepath = home_dir+f"data/dbsnp/reference_snps/{rs_id}.json"
with open(inp_filepath, 'r') as f:
    snp_json_obj = json.load(f)

hgvs_reports = snp_json_obj["primary_snapshot_data"]["placements_with_allele"] 
# print(hgvs_reports)
for report in hgvs_reports:
    if report["placement_annot"]["mol_type"] == "protein": 
        prot_isoforms = [allele["hgvs"] for allele in report["alleles"]]
        print(prot_isoforms)
            


['NP_004151.4:p.Arg37=', 'NP_004151.4:p.Arg37Cys', 'NP_004151.4:p.Arg37Gly', 'NP_004151.4:p.Arg37Ser']
['NP_001380958.1:p.Arg37=', 'NP_001380958.1:p.Arg37Cys', 'NP_001380958.1:p.Arg37Gly', 'NP_001380958.1:p.Arg37Ser']
['NP_001380957.1:p.Arg37=', 'NP_001380957.1:p.Arg37Cys', 'NP_001380957.1:p.Arg37Gly', 'NP_001380957.1:p.Arg37Ser']
